# Python Course - Tutorial 8

### Exercise 1 (Data Visualization with Matplotlib)

Consider the music.db database provided in our GitHub repository. Create a histogram
that shows the distribution of the total number of items purchased per invoice. Add
appropriate axis labels, a title, and a grid to the plot. Save the plot in a designated
directory named images/ and exclude this directory from version control.

In [ ]:
import pathlib
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to the SQLite database and read the 'invoice_items' table into a DataFrame
with sqlite3.connect("../music.db") as conn:
    invoices_items_df = pd.read_sql_query("SELECT * FROM invoice_items", conn)

# Group the data by 'InvoiceId' and calculate the total quantity of items per invoice
invoice_totals = invoices_items_df.groupby("InvoiceId")["Quantity"].sum()

# Plot a histogram of the total quantities of items per invoice
plt.hist(invoice_totals, bins=20)
plt.xlabel("Total Quantity of Items per Invoice")
plt.ylabel("Frequency")
plt.title("Distribution of Invoice Quantities")
plt.grid(True)

# Create an 'images' directory if it doesn't already exist and save the plot
pathlib.Path("images").mkdir(exist_ok=True)
plt.savefig("images/invoice_quantity_histogram.jpg")

# Display the histogram plot
plt.show()


### Exercise 2 (Linear Regression with Statsmodels)

Using the `music.db` database, investigate the relationship between track length and the number of times a track has been streamed. Treat track length (in milliseconds) as the **independent variable** and the total number of streams (from the play histories table) as the **dependent variable**.

*Note:* Since track length does not have mean zero, you should include an intercept in the model. For this, you can use `X = sm.add_constant(X, prepend=True)` before fitting the model.


In [ ]:
import statsmodels.api as sm
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Connect to the SQLite database and fetch data from 'tracks' and 'play_histories' tables
conn = sqlite3.connect("../music.db")
tracks_df = pd.read_sql_query("SELECT TrackId, Milliseconds FROM tracks", conn)
play_histories_df = pd.read_sql_query("SELECT * FROM play_histories", conn)
conn.close()

# Aggregate the number of streams for each track
stream_counts = play_histories_df.groupby("TrackId").size().reset_index(name="NumStreams")

# Merge the track data with the stream counts on 'TrackId'
merged_df = pd.merge(tracks_df, stream_counts, on="TrackId", how="inner")

# Prepare the independent variable (track length) and dependent variable (number of streams)
X = merged_df["Milliseconds"].astype(float)
y = merged_df["NumStreams"].astype(float)
X = sm.add_constant(X, prepend=True)  # Add a constant term for the regression model

# Fit a linear regression model using Ordinary Least Squares (OLS)
model = sm.OLS(y, X).fit()
print(model.summary())

# Plot the data and the regression line
plt.scatter(merged_df["Milliseconds"], merged_df["NumStreams"], label="Data", alpha=0.7)
plt.plot(merged_df["Milliseconds"], model.predict(X), color="red", label="Fit")

# Label the axes, add a title, grid, and legend to the plot
plt.xlabel("Track Length (ms)")
plt.ylabel("Number of Streams")
plt.title("Linear Regression: Number of Streams vs. Track Length")
plt.grid()
plt.legend()
plt.show()


### Exercise 3 (Statistical Analysis with SciPy)

The `northwind.db` database from our GitHub repository (`tutorials/data/northwind.db`) contains unit price and category information for various products. Write a function that takes two category names as input and performs a **t-test** to determine if the mean product unit prices of the two categories are significantly different. The function should return a boolean value indicating whether the difference is significant at a **95% confidence level** (i.e., whether the p-value is less than 0.05).


In [ ]:
import sqlite3
import pandas as pd
from scipy import stats

def has_significant_price_differences(category1, category2):
    # Connect to the SQLite database and fetch data from 'Products' and 'Categories' tables
    with sqlite3.connect("data/northwind.db") as conn:
        products_df = pd.read_sql_query("SELECT * FROM Products", conn)
        categories_df = pd.read_sql_query("SELECT * FROM Categories", conn)

    # Merge the products and categories data on 'CategoryID'
    merged_df = pd.merge(products_df, categories_df, on="CategoryID", how="inner")

    # Extract prices for the specified categories
    cat1_prices = merged_df.loc[merged_df["CategoryName"] == category1, "UnitPrice"]
    cat2_prices = merged_df.loc[merged_df["CategoryName"] == category2, "UnitPrice"]

    # Perform an independent t-test to compare the two categories
    t_stat, p_val = stats.ttest_ind(cat1_prices, cat2_prices)

    # Print the test statistics and interpret the result
    print("T-statistic:", round(t_stat, 4))
    print("P-value:", round(p_val, 4))
    if p_val < 0.05:
        print("Reject null hypothesis: The means are different")
        return True
    else:
        print("Fail to reject null hypothesis: The means are the same")
        return False

# Check for significant price differences between two categories
result = has_significant_price_differences("Beverages", "Condiments")
print(result)


### Exercise 4 (Optional: Additional Python Resources for Data Science)

To further strengthen your skills in Python for data analysis and visualization, consider completing the following tasks:

1. **Discover Additional Resources for Popular Python Packages**:  
   - Read parts of [Python for Data Analysis](https://wesmckinney.com/book/) by Wes McKinney to enhance your knowledge of Pandas and NumPy for data manipulation.  
   - Check out [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) by Jake VanderPlas, which covers essential tools like Matplotlib, SciPy, and Scikit-learn.  
   - Review the [Matplotlib Cookbook](https://github.com/rougier/scientific-visualization-book) for hands-on examples of data visualization with Matplotlib.  
   - Explore the documentation for [Seaborn](https://seaborn.pydata.org/) (for advanced statistical plots) and [Plotly](https://plotly.com/python/) (for interactive, web-based visualizations).  
   - Visit [Claus Wilke's Data Visualization](https://clauswilke.com/dataviz/) for advice on creating effective visualizations.  
   - Dive into the [Scientific Python Lectures](https://lectures.scientific-python.org/) to deepen your understanding of Python in scientific computing.  

2. **Practice and Apply Your Skills**:  
   - Challenge yourself with Python exercises using NumPy and Pandas on platforms like [StrataScratch](https://www.stratascratch.com), which offers real-world datasets and interview questions.  
   - Explore practical problems and datasets on [Kaggle](https://www.kaggle.com).  

3. **Choose Your Own Project**:  
   - Alternatively, pick a project or topic that interests you and apply the concepts you've learned to it.
